<a href="https://colab.research.google.com/github/bryaanabraham/Wandering_Guardian/blob/main/ML%20Implementation/ML_Navigation_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np

In [33]:
coords = pd.read_csv("training_dataset.csv")
coords.head()

,longitude,latitude,path_id
0,12.821062,80.044729,1
1,12.821219,80.044743,1
2,12.821479,80.044736,1
3,12.821561,80.044743,1
4,12.821636,80.044743,1


In [34]:
# Deviation parameters
min_deviation_distance = -0.00004
max_deviation_distance = 0.00004
#deviation is set to zero for now so elevation remains constant
max_deviation_angle = 0

# Generate random deviation distances and angles for each point in the DataFrame
num_points = len(coords)
random_deviation_distances = np.random.uniform(min_deviation_distance, max_deviation_distance, num_points)
random_deviation_angles = np.random.uniform(0, max_deviation_angle, num_points)

# Convert random angles to radians
random_deviation_angles_radians = np.radians(random_deviation_angles)

# Calculate deviation in latitude and longitudet
deviation_distance_lat = random_deviation_distances * np.cos(random_deviation_angles_radians)
deviation_distance_lon = random_deviation_distances * np.sin(random_deviation_angles_radians)

decoy_coords = coords.copy()
decoy_coords[' latitude'] += deviation_distance_lat
decoy_coords['longitude'] += deviation_distance_lon

decoy_coords.head()

,longitude,latitude,path_id
0,12.821062,80.044741,1
1,12.821219,80.044755,1
2,12.821479,80.044717,1
3,12.821561,80.044745,1
4,12.821636,80.044718,1


In [35]:
decoy_coords.head()

,longitude,latitude,path_id
0,12.821062,80.044741,1
1,12.821219,80.044755,1
2,12.821479,80.044717,1
3,12.821561,80.044745,1
4,12.821636,80.044718,1


In [36]:
print('coords keys:', coords.keys())
print('decoy_coords keys:', decoy_coords.keys())

coords keys: Index(['longitude', ' latitude', ' path_id'], dtype='object')
decoy_coords keys: Index(['longitude', ' latitude', ' path_id'], dtype='object')


In [37]:
import folium

m = folium.Map()

gps_path = list(zip(decoy_coords['longitude'], decoy_coords[' latitude']))
folium.PolyLine(locations=gps_path, color='red', weight=2.5, opacity=0.8).add_to(m)
gps_path = list(zip(coords['longitude'], coords[' latitude']))
folium.PolyLine(locations=gps_path, color='blue', weight=2.5, opacity=0.8).add_to(m)

max_lat = max(coords['longitude'])
min_lat = min(coords['longitude'])
min_lon = min(coords[' latitude'])
max_lon = max(coords[' latitude'])
m.fit_bounds([(min_lat, min_lon), (max_lat, max_lon)])

m

In [47]:
from geopy.distance import geodesic

m = folium.Map()

gps_path = list(zip(decoy_coords['longitude'], decoy_coords[' latitude']))
folium.PolyLine(locations=gps_path, color='red', weight=2.5, opacity=0.8).add_to(m)
gps_path = list(zip(coords['longitude'], coords[' latitude']))
folium.PolyLine(locations=gps_path, color='blue', weight=2.5, opacity=0.8).add_to(m)

max_lat = max(coords['longitude'])
min_lat = min(coords['longitude'])
min_lon = min(coords[' latitude'])
max_lon = max(coords[' latitude'])
m.fit_bounds([(min_lat, min_lon), (max_lat, max_lon)])

def calculate_deviation(coord1, coord2):
    return geodesic(coord1, coord2).kilometers

deviations = []

# Iterate through every 100th index and calculate the distance between coords and decoy_coords
for i in range(0, min(len(coords), len(decoy_coords)), 5):
    deviation = calculate_deviation((coords.iloc[i]['longitude'], coords.iloc[i][' latitude']),
                                  (decoy_coords.iloc[i]['longitude'], decoy_coords.iloc[i][' latitude']))
    deviations.append((i, deviation))

for index, deviation in deviations:
    coord = (decoy_coords.iloc[index]['longitude'], decoy_coords.iloc[index][' latitude'])
    if deviation <= 0.002:
      colour = 'green'
    elif deviation >= 0.004:
      colour = 'red'
    else:
      colour = 'orange'
    m.add_child(folium.Marker(location=coord,
                              popup=f"Deviation: {deviation * 1000:.2f} m",
                              icon=folium.Icon(color=colour)))
m